In [1]:
import pandas as pd
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

In [10]:
# 加载本地模型路径，请换成你实际下载的路径
# LLM 路径
llm_model_path = "Qwen/Qwen2.5-1.5B-Instruct"
# Embedding 模型路径
embedding_model_path = "sentence-transformers/all-mpnet-base-v2"

In [11]:

# 加载嵌入模型
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [16]:

# 加载CSV数据
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path="D:\ClassMaterial\graphrag\data_test\检验信息.csv", encoding='GB2312')
documents = loader.load()


In [17]:
for doc in documents[:5]:  # 只打印前5个文档作为示例
    print(doc.page_content)

患者就诊卡证号码: D8E0B803CA9AF9033BD5A321977B9016984A353C7D4C5488
门诊急诊住院体检标识: 1
门诊急诊住院体检编号: 7EBB94CF52DEF7CB26655189350E54B578D9B778DB6635AC8CB5A4062BA44281
检验申请单编号: 2B8CAE5C0FFD797379B004D8215931B6EB0A6CF9AA036C71
样品分类代码: 9999
样品分类名称: 未知
标本采样时间: 2016/11/2
采样单位编码: 
检验报告编号: 2B8CAE5C0FFD797379B004D8215931B6EB0A6CF9AA036C71
检验报告名称: 尿常规检查(尿液分析)
检验项目标准编码: 80302000
检验结果: 阴性
检验结果计量单位: 
检验结果是否异常: 0
检验方法: 
检验科室编码: A30
检验科室名称: 检验科
检验日期和时间: 2016/11/2
患者就诊卡证号码: D8E0B803CA9AF9033BD5A321977B9016984A353C7D4C5488
门诊急诊住院体检标识: 1
门诊急诊住院体检编号: 7EBB94CF52DEF7CB26655189350E54B578D9B778DB6635AC8CB5A4062BA44281
检验申请单编号: 2B8CAE5C0FFD797379B004D8215931B6EB0A6CF9AA036C71
样品分类代码: 9999
样品分类名称: 未知
标本采样时间: 2016/11/2
采样单位编码: 
检验报告编号: 2B8CAE5C0FFD797379B004D8215931B6EB0A6CF9AA036C71
检验报告名称: 尿常规检查(尿液分析)
检验项目标准编码: 80601000
检验结果: 黄
检验结果计量单位: 
检验结果是否异常: 0
检验方法: 
检验科室编码: A30
检验科室名称: 检验科
检验日期和时间: 2016/11/2
患者就诊卡证号码: D8E0B803CA9AF9030016738B3CA923096DC7D3DFFFEE861F
门诊急诊住院体检标识: 1
门诊急诊住院体检编号: 8687B660A14CD5A5C0A24FFF40C059A188764AC156

In [20]:
# 构建FAISS索引
text = [doc.page_content for doc in documents]
print(text[:5])  # 只打印前5个文档作为示例

['患者就诊卡证号码: D8E0B803CA9AF9033BD5A321977B9016984A353C7D4C5488\n门诊急诊住院体检标识: 1\n门诊急诊住院体检编号: 7EBB94CF52DEF7CB26655189350E54B578D9B778DB6635AC8CB5A4062BA44281\n检验申请单编号: 2B8CAE5C0FFD797379B004D8215931B6EB0A6CF9AA036C71\n样品分类代码: 9999\n样品分类名称: 未知\n标本采样时间: 2016/11/2\n采样单位编码: \n检验报告编号: 2B8CAE5C0FFD797379B004D8215931B6EB0A6CF9AA036C71\n检验报告名称: 尿常规检查(尿液分析)\n检验项目标准编码: 80302000\n检验结果: 阴性\n检验结果计量单位: \n检验结果是否异常: 0\n检验方法: \n检验科室编码: A30\n检验科室名称: 检验科\n检验日期和时间: 2016/11/2', '患者就诊卡证号码: D8E0B803CA9AF9033BD5A321977B9016984A353C7D4C5488\n门诊急诊住院体检标识: 1\n门诊急诊住院体检编号: 7EBB94CF52DEF7CB26655189350E54B578D9B778DB6635AC8CB5A4062BA44281\n检验申请单编号: 2B8CAE5C0FFD797379B004D8215931B6EB0A6CF9AA036C71\n样品分类代码: 9999\n样品分类名称: 未知\n标本采样时间: 2016/11/2\n采样单位编码: \n检验报告编号: 2B8CAE5C0FFD797379B004D8215931B6EB0A6CF9AA036C71\n检验报告名称: 尿常规检查(尿液分析)\n检验项目标准编码: 80601000\n检验结果: 黄\n检验结果计量单位: \n检验结果是否异常: 0\n检验方法: \n检验科室编码: A30\n检验科室名称: 检验科\n检验日期和时间: 2016/11/2', '患者就诊卡证号码: D8E0B803CA9AF9030016738B3CA923096DC7D3DFFFEE861F\n门诊急诊住院体检标识: 1\n门诊急诊住院体检编号

In [22]:
vectorstore = FAISS.from_texts(text, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})  # 检索最相关的2个文档

In [23]:
# 加载本地大语言模型
tokenizer = AutoTokenizer.from_pretrained(llm_model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    llm_model_path,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [24]:
# 创建文本生成管道
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=200,
    temperature=0.7,
    repetition_penalty=1.1,
    return_full_text=False
)

Device set to use cuda:0


In [25]:
# 包装成 LangChain 可用的 LLM
llm = HuggingFacePipeline(pipeline=pipe)

# 定义提示模板
prompt_template = """你是一个智能助手，请根据检索到的信息回答用户问题。
Context: {context}
Question: {question}
Answer:"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# 创建 QA 链
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

# 封装一下查询函数
def query_rag(question):
    result = qa_chain.invoke({"query": question})
    answer = result["result"]
    source_docs = "\n".join([doc.page_content for doc in result["source_documents"]])
    return answer, source_docs

C:\Users\12873\AppData\Local\Temp\ipykernel_19536\294432507.py:2: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [27]:
# 测试查询功能
question = "患者就诊卡号码为D8E0B803CA9AF9033BD5A321977B9016984A353C7D4C5488的病人所有检验时间是多少？"
answer, sources = query_rag(question)
print(f"答案：{answer}\n来源：{sources}")

答案： 根据提供的信息，该患者的检验报告记录显示了两次不同的检验时间：

第一次检验时间为：2022年12月3日10:14:00
第二次检验时间为：2022年5月6日

所以，这位患者的所有检验时间分别是2022年12月3日10:14:00和2022年5月6日。
来源：患者就诊卡证号码: 8A5CAD8B3C86B4959766C3B2D2382D1ECF97A55365E150ED
门诊急诊住院体检标识: 3
门诊急诊住院体检编号: 9B55B8098BB8E50F4CCAB6B35224A4D5
检验申请单编号: 82C1BEBAA801EF53B42CD01517BA7C44
样品分类代码: 0102
样品分类名称: 血清
标本采样时间: 2022/12/3 8:07:22
采样单位编码: 320113466002673
检验报告编号: 82C1BEBAA801EF53B42CD01517BA7C44
检验报告名称: 男性肿标全套（免疫A）
检验项目标准编码: 80403000
检验结果: 6.710
检验结果计量单位: U/ml
检验结果是否异常: 0
检验方法: 
检验科室编码: A30
检验科室名称: 检验科
检验日期和时间: 2022/12/3 10:14:00
患者就诊卡证号码: 8A5CAD8B3C86B4959766C3B2D2382D1ECF97A55365E150ED
门诊急诊住院体检标识: 3
门诊急诊住院体检编号: A7DF3289527A0A176BFAC6DB48345A09
检验申请单编号: D2D92CD36338A0502AA064556542E9B3
样品分类代码: 0102
样品分类名称: 血清
标本采样时间: 2022/5/6
采样单位编码: 320113466002673
检验报告编号: D2D92CD36338A0502AA064556542E9B3
检验报告名称: 男性肿标全套（免疫A）
检验项目标准编码: 80403000
检验结果: 3.280
检验结果计量单位: U/ml
检验结果是否异常: 0
检验方法: 
检验科室编码: A30
检验科室名称: 检验科
检验日期和时间: 2022/5/6


In [31]:
# Gradio 接口函数
import gradio as gr
def rag_query_interface(question):
    answer, sources = query_rag(question)
    return f"**答案：**\n\n{answer}\n\n**来源信息：**\n\n{sources}"

# 创建 Gradio 界面
iface = gr.Interface(
    fn=rag_query_interface,
    inputs=gr.Textbox(lines=2, placeholder="请输入你的问题..."),
    outputs=gr.Markdown(label="回答与来源"),
    title="RAG 中文问答系统",
    description="基于本地 CSV 数据 + Qwen2.5-1.5B-Instruct + all-mpnet-base-v2 的 RAG 模型问答系统。",
    examples=[["患者就诊卡号码为D8E0B803CA9AF9033BD5A321977B9016984A353C7D4C5488的病人所有检验时间是什么"], ["患者就诊卡号码为D8E0B803CA9AF9033BD5A321977B9016984A353C7D4C5488的病人检验了哪些项目"]],
)

# 启动服务
if __name__ == "__main__":
    iface.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
